ПРОСМОТРЕТЬ РАЗБОР БОЛЬШЕГО КОЛИЧЕСТВА ТЕКСТОВ
ПОСМОТРЕТЬ ГДЕ КОНТЕКСТ РЕАЛЬНО ВЫРУЧАЕТ А ГДЕ НЕТ

In [ ]:
учитывает tf idf и расстояние

In [ ]:
def get_dependencies (conllu_map, text_map_input):
    assert len(conllu_map) == len(text_map_input) #sentences count is equal
    text_map = copy.deepcopy(text_map_input)
    for sentence_conllu, text_map_sentence in zip(conllu_map,text_map):
        dep_dict = {}#включаем зависимости неисключенных слов от всех слов(если исключенные будут с контекстом то их просто оштрафует и нам не страшно)
        #print(sentence_conllu)
        for word_conllu,word_map in zip(sentence_conllu,text_map_sentence): 
            #print(word_conllu)
            #print(word_map,'\n')
            #print(word[1], "head_word_nominal_index =", word[6])
            if (word_conllu[3] not in pos_exclude_list):#make sure that dependent list will not count exclude pos
                if(word_conllu[6] in dep_dict):
                    #print(word_conllu)
                    dep_dict[word_conllu[6]].append((word_map['vocabulary_prop']['tf_idf'],word_map['vocabulary_prop']['real_index']))
                elif(int(word_conllu[6]) != 0):
                    dep_dict[word_conllu[6]] = []
                    dep_dict[word_conllu[6]].append((word_map['vocabulary_prop']['tf_idf'],word_map['vocabulary_prop']['real_index']))
        #print(dep_dict,'\n')
        print("\n====\n",text_map_sentence,"====\n")
        for map_word in text_map_sentence:
            word_nominal_index = map_word["vocabulary_prop"]["nominal_index"] 
            if(word_nominal_index in dep_dict):#calculate context fine
                print(map_word)
                tf_idf_per_dist_sum = 0 
                word_real_index = map_word["vocabulary_prop"]["real_index"]
                print(dep_dict[word_nominal_index])
                for dependent_element in dep_dict[word_nominal_index]:
                    distance = abs(word_real_index - dependent_element[1])
                    print("distance", distance)
                    dep_element_adjusted_tfidf = dependent_element[0] + 1
                    print("dep_element_adjusted_tfidf",dep_element_adjusted_tfidf)
                    tf_idf_per_dist_element = dep_element_adjusted_tfidf/distance
                    print("tf_idf_per_dist_element",tf_idf_per_dist_element)
                    tf_idf_per_dist_sum += tf_idf_per_dist_element
                    print("tf_idf_per_dist_sum",tf_idf_per_dist_sum)
                context_fine = 1 / tf_idf_per_dist_sum
                print("fine", context_fine)
                print("ADJUSTED tfidf for word ",map_word['word'], map_word["vocabulary_prop"]["tf_idf"] * context_fine)
                map_word["vocabulary_prop"]["vocab_importane"] = map_word["vocabulary_prop"]["tf_idf"] * context_fine
            else:
                map_word["vocabulary_prop"]["vocab_importane"] = map_word["vocabulary_prop"]["tf_idf"]
        print()
        """
        for map_word in text_map_sentence:
            if(map_word["vocabulary_prop"]["nominal_index"] in dep_dict):
                map_word["vocabulary_prop"]["dep_words_count"] = dep_dict[map_word["vocabulary_prop"]["nominal_index"]] + 1
                map_word["vocabulary_prop"]["vocab_importane"] = map_word["vocabulary_prop"]["dep_words_count"] * map_word["vocabulary_prop"]["tf_idf"]
            else:
                map_word["vocabulary_prop"]["dep_words_count"] = 1
                map_word["vocabulary_prop"]["vocab_importane"] = map_word["vocabulary_prop"]["dep_words_count"] * map_word["vocabulary_prop"]["tf_idf"]
            """
    
    return text_map
text_map_dep = get_dependencies(conllu_text_map_ex, text_map_ex)

In [ ]:
С ЛОГРАИФМАМИ БЕЗ ТФ ИДФ

In [ ]:
minimal_distances = []
min_current_distance = 1
min_dist_dict = {}
for i in range (1,15):
    #print(i)
    minimal_distances.append(min_current_distance)
    min_dist_dict [i] = copy.deepcopy(minimal_distances)
    if(i%2 == 0):
        min_current_distance += 1
        #print("herak",min_current_distance)
    #print(min_dist_dict)

In [ ]:
mean_min_dist = {}
for key,values in  min_dist_dict.items():
    #print(key, values, mean(values))
    mean_min_dist[key] = mean(values)
mean_min_dist

In [ ]:
pos_exclude_list = ["AUX","SYM","CCONJ","X","DET","NUM","PART","SCON","INTJ","PROPN"]

In [ ]:
не общее количество слов в подчинении а количество слов в финальном подчинении (таких у которх нет относящихся к ним слов) -- не норм
слова в предпоследнем уровне обесценятся

ВАРИАНТ 1)
Лучше просто исключить из списка заивсимых все малозначимые слова типа Closed class words + Other из (кроме ADP) http://universaldependencies.org/u/pos/index.html

ВАРИАНТ 2
ДУМАТЬ О ЗАДАЧЕ ПРЕДСКАЗАНИЯ ВЕРОЯТНОСТИ ДОГАДКИ О СМЫСЛЕ СЛОВА ИЗ ЕГО КОНТЕКСТА!!
ДЕЛАТЬ НАОБОРОТ
ЧЕМ БОЛШЕ У СЛОВА ПРЯМО СВЯЗАННЫХ СЛОВ ТЕМ БОЛЬШЕ ВЕРОЯТНОСТЬ ЧТО О ЕГО ЗНАЧЕНИИ МОЖНО ДОГАДАТЬСЯ ТЕМ МЕНЬШЕ ВЕРОЯТНОСТЬ ТОГО ЧТО ИМЕННО ЭТО СЛОВО УБЬЕТ ВСЕ ПОНИМАНИЕ ПРЕДЛОЖЕНИЯ

В то же время из контекста надо исключить тот же список 

Итого штрафуем важность слова в совокупном понимании предложения на величину 1/(кол-во прямо зависимых слов)


In [ ]:
def get_dependencies (conllu_map, text_map_input, debug = False):
    assert len(conllu_map) == len(text_map_input) #sentences count is equal
    text_map = copy.deepcopy(text_map_input)
    for sentence_conllu, text_map_sentence in zip(conllu_map,text_map):
        nominal_index_correction_dict = {}
        dep_dict = {}#включаем зависимости неисключенных слов от всех слов(если исключенные будут с контекстом то их просто оштрафует и нам не страшно)
        #print(sentence_conllu)
        nom2real_ind = {}
        for word_conllu,word_map in zip(sentence_conllu,text_map_sentence): 
            nom2real_ind[word_conllu[0]] = word_map['vocabulary_prop']['real_index']
            #print(word_conllu)
            #print(word_map,'\n')
            #print(word[1], "head_word_nominal_index =", word[6])
            if (word_conllu[3] not in pos_exclude_list):#make sure that dependent list will not count exclude pos
                if(word_conllu[6] in dep_dict):
                    dep_dict[word_conllu[6]].append(word_map['vocabulary_prop']['real_index'])
                elif(int(word_conllu[6]) != 0):
                    dep_dict[word_conllu[6]] = []
                    dep_dict[word_conllu[6]].append(word_map['vocabulary_prop']['real_index'])
            else:
                nominal_index_correction_dict[word_conllu[0]] = (0.5,"excluded_pos")
        if (debug == True):
            print("sentence_conllu", sentence_conllu)            
            print("dep_dict", dep_dict)
            print("nom2real_ind",nom2real_ind)
        real_mean_dist = {}
        
        for head_element_nom_index, dep_elt_real_indexes in  dep_dict.items():
            distances = []
            
            for dep_el_r_i in dep_elt_real_indexes:
                if (debug == True):
                    print("dep_el_r_i",dep_el_r_i)
                    print("distance", abs(dep_el_r_i - nom2real_ind[head_element_nom_index]))
                distances.append(abs(dep_el_r_i - nom2real_ind[head_element_nom_index]))

            real_mean_distance = mean(distances)
            real_mean_dist[head_element_nom_index] = real_mean_distance
            real_context_element_count = len(dep_elt_real_indexes)
            minimal_mean_distance = mean_min_dist[real_context_element_count]
            
            if real_context_element_count == 1: real_context_element_count += 1
            real_context_mean_distance = real_mean_dist[head_element_nom_index]
            
            correction_coeff = (1+ np.log10(real_context_mean_distance/minimal_mean_distance))/(1+np.log(real_context_element_count))
            if (debug == True):
                print("mean_min_dist with",real_context_element_count, "elements", minimal_mean_distance)
                print("real_mean_dist",real_context_mean_distance)
                print("correction_coeff",correction_coeff,"vs", 1/real_context_element_count,"without log")
            nominal_index_correction_dict[head_element_nom_index] = [correction_coeff,dep_elt_real_indexes]
            
        #ШТРАФОВАТЬ ВАЖНОСТЬ ИСКЛЮЧАЕМЫХ СЛОВ 
        
        
        #print("\n====\n",text_map_sentence,"====\n")
        for map_word in text_map_sentence:
            word_nominal_index = map_word["vocabulary_prop"]["nominal_index"] 
            if(word_nominal_index in nominal_index_correction_dict):#calculate context fine
                print(nominal_index_correction_dict[word_nominal_index])
                map_word["vocabulary_prop"]["dep_element_real_ind"] = nominal_index_correction_dict[word_nominal_index][1]
                map_word["vocabulary_prop"]["context_correction"] = nominal_index_correction_dict[word_nominal_index][0]
                map_word["vocabulary_prop"]["vocab_importance"] = nominal_index_correction_dict[word_nominal_index][0] * map_word["vocabulary_prop"]['tf_idf'] 
            else:
                map_word["vocabulary_prop"]["vocab_importance"] = map_word["vocabulary_prop"]["tf_idf"]
    return text_map
                
get_dependencies(conllu_text_map_ex, text_map_ex, debug = True) 


